In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

### **Data Reading**

In [0]:
df = spark.read.format("parquet") \
          .load("abfss://bronze@databricksend2end.dfs.core.windows.net/customers")

In [0]:
display(df)

In [0]:
df = df.drop("_rescued_data")
display(df)

In [0]:
df = df.withColumn("domains",split(col("email"),"@")[1])
display(df)

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort(desc("total_customers")).display()

In [0]:
df_gmail = df.filter(col("domains")=="gmail.com")
df_gmail.display()

In [0]:
df_gmail = df.filter(col("domains")=="gmail.com")
df_gmail.display()
time.sleep(5)

df_yahoo = df.filter(col("domains")=="yahoo.com")
df_yahoo.display()
time.sleep(5)

df_hotmail = df.filter(col("domains")=="hotmail.com")
df_hotmail.display()
time.sleep(5)

In [0]:
df = df.withColumn("full_name", concat(col("first_name"), lit(" "), col("last_name")))
# df.withColumn("full_name", concat_ws(" ", col("first_name"), col("last_name"))).display()
df = df.drop("first_name", "last_name")
display(df)

### **Data Writing**

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@databricksend2end.dfs.core.windows.net/customers")

In [0]:
# %sql

# CREATE SCHEMA databricks_cata.silver

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.customers
USING DELTA
LOCATION 'abfss://silver@databricksend2end.dfs.core.windows.net/customers'

In [0]:
%sql

select * from databricks_cata.silver.customers